# This is the tag along code for the stage C quiz

### Author details
#### Name: Jimoh Abdulsomad Abiola
#### Hamoye ID:  15e47f79f981f000

#### Importing the necessary modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
import xgboost as xgb
import lightgbm as lgb

In [ ]:
data = pd.read_csv("Data_for_UCI_named.csv")

In [ ]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [ ]:
TP=255 
TN=20
FP=1380
FN=45

precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1_score = 2*(precision*recall)/(precision+recall)
f1_score

0.2635658914728682

In [ ]:
data.drop(columns = ['stab'],inplace = True)

In [ ]:
x = data.drop(columns = 'stabf')
y = data['stabf']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1)
rfc.fit(x_train, y_train)

pred_rfc = rfc.predict(x_test)
acc_rfc = accuracy_score(y_test,pred_rfc)
acc_rfc

0.929

In [ ]:
import  xgboost as xgb
model_xgb = xgb.XGBClassifier(random_state=1)
model_xgb.fit(x_train, y_train)
pred_xgb = model_xgb.predict(x_test)
acc_xgb = accuracy_score(y_test,pred_xgb)
acc_xgb

0.9195

In [ ]:
import lightgbm as lgb
model_lgb = lgb.LGBMClassifier(random_state=1)
model_lgb.fit(x_train,y_train)
pred_lgb = model_lgb.predict(x_test)
acc_lgb = accuracy_score(y_test,pred_lgb)
acc_lgb

0.9375

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state=1)
etc.fit(x_train, y_train)

pred_etc = etc.predict(x_test)
acc_etc = accuracy_score(y_test,pred_etc)
acc_etc

0.928

In [ ]:
hyperparam = {
              'n_estimators':[100,500,300,1000],
              'min_samples_split':[7,2,5,2],
              'min_samples_leaf':[4,8,6,8],
              'max_features':[None,'log2','auto',None]
              }
from sklearn.model_selection import RandomizedSearchCV
randomcv = RandomizedSearchCV(etc,hyperparam,cv=5,n_iter = 10,scoring='accuracy',n_jobs=-1,verbose=1,random_state=1)
searchcv = randomcv.fit(x_train,y_train)
searchcv.best_params_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 500,
 'min_samples_split': 5,
 'min_samples_leaf': 6,
 'max_features': None}

In [ ]:
new_etc = ExtraTreesClassifier(n_estimators=500,min_samples_split=5,min_samples_leaf=6,max_features=None,random_state=1)
new_etc.fit(x_train,y_train)
pred_netc=new_etc.predict(x_test)
acc_netc = accuracy_score(y_test,pred_netc)
acc_netc

0.9295

In [ ]:
feat_imp = new_etc.feature_importances_
cols = x.columns
df = pd.DataFrame({
    "features":cols,
    "feature_importances":feat_imp
}).sort_values(by="feature_importances")
df

,features,feature_importances
4,p1,0.005458
7,p4,0.006922
6,p3,0.007216
5,p2,0.007452
8,g1,0.103252
9,g2,0.107721
11,g4,0.108954
10,g3,0.112378
2,tau3,0.133285
3,tau4,0.133771


tau2 and p1